In [1]:
import findspark
findspark.init()

In [2]:
import pyspark
findspark.find()

'/opt/spark'

# 1. RDD 를 사용하기 위해서는 스파크 컨텍스트 생성이 필요

In [3]:
from pyspark import SparkContext, SparkConf

In [4]:
conf = SparkConf().setMaster("local").setAppName("Spark Practice")

In [5]:
sc = SparkContext(conf=conf)

# 2. RDD 생성하기

In [6]:
data = sc.parallelize([ 
         ('Ferrari', 'fast'),
         {'porshe' : 100000},
         ['spain', 'visited, 4504']])

In [7]:
data

ParallelCollectionRDD[0] at parallelize at PythonRDD.scala:195

In [12]:
dir(data)

['__add__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getnewargs__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_computeFractionForSampleSize',
 '_defaultReducePartitions',
 '_id',
 '_is_barrier',
 '_jrdd',
 '_jrdd_deserializer',
 '_memory_limit',
 '_pickled',
 '_reserialize',
 '_to_java_object_rdd',
 'aggregate',
 'aggregateByKey',
 'barrier',
 'cache',
 'cartesian',
 'checkpoint',
 'coalesce',
 'cogroup',
 'collect',
 'collectAsMap',
 'combineByKey',
 'context',
 'count',
 'countApprox',
 'countApproxDistinct',
 'countByKey',
 'countByValue',
 'ctx',
 'distinct',
 'filter',
 'first',
 'flatMap',
 'flatMapValues',
 'fold',
 'foldByKey',
 'foreach',
 'foreachPartition',
 'fullOuterJoin',
 'getCheckpo

# 3. 액션 실행하기

      - 액션을 실행하면 RDD와 다른 자료형으로 변환

In [10]:
data_ = data.collect()

In [11]:
data_

[('Ferrari', 'fast'), {'porshe': 100000}, ['spain', 'visited, 4504']]

In [10]:
data_[1]['porshe']

100000

# 4.   파일 데이터 가져오기

     - 실제 파일 가져오기는 action 이 실행될 때 처리된다. 
     - 예외가 발생시 데이터를 제대로 처리하는 지를 알아본다. 

In [13]:
%%writefile abc.txt
은옥주
조현웅
정승호

Writing abc.txt


In [16]:
data_from_file = sc.textFile('./abc.txt')

In [17]:
data_from_file

./abc.txt MapPartitionsRDD[4] at textFile at NativeMethodAccessorImpl.java:0

In [18]:
data_from_file.collect()

['은옥주', '조현웅', '정승호']

In [19]:
data_from_file.first()

'은옥주'

In [22]:
data_from_file.take(2)

['은옥주', '조현웅']

In [11]:
#data_from_file = sc.textFile('/Users/dahlmoon/Documents/data/VS14MORT.txt')

In [24]:
!ls 

1910.13233.pdf
Learning-PySpark.pdf
RDD 생성하기-Copy1.ipynb
RDD 생성하기.ipynb
README.md
Untitled.ipynb
Untitled1.ipynb
Untitled2.ipynb
abc.txt
data
data_key.txt
employee.json
employee.parquet
foreach.py
learningPySpark-master
people.json
people.txt
pyspark _윈도우즈 스파크 설치 및 기본.ipynb
pyspark 기본처리_1111.ipynb
pyspark.pdf
pyspark_2.pdf
pyspark_DataFrame 만들고 조회하기.ipynb
pyspark_JSON_Parquet파일처리하기.ipynb
pyspark_RDD 기본처리 알아보기.ipynb
pyspark_Row 클래스로 객체 만들기.ipynb
pyspark_SQL 이해하기.ipynb
pyspark_데이터프레임 기초 이해하기.ipynb
requirements.txt
spark-warehouse
스파크 컨텍스트와 세션 이해하기.ipynb
파이스파크  알아보기.ipynb
파이스파크_json데이터 읽어오기.ipynb
파이스파크_시작과 종료하는방법.ipynb


## 데이터 다운로드

Note, that to execute the code above you will have to change the path where the data is stored. 
The dataset can be downloaded from 
http://tomdrabas.com/data/VS14MORT.txt.gz

In [13]:
data_from_file

/Users/dahlmoon/Documents/data/VS14MORT.txt MapPartitionsRDD[2] at textFile at NativeMethodAccessorImpl.java:0

In [14]:
data_from_file.take(1)

['                   1                                          2101  M1087 432311  4M4                2014U7CN                                    I64 238 070   24 0111I64                                                                                                                                                                           01 I64                                                                                                  01  11                                 100 601']

## 텍스트 문서에 대한 포매팅하기

In [15]:
def extractInformation(row):
    import re
    import numpy as np

    selected_indices = [
         2,4,5,6,7,9,10,11,12,13,14,15,16,17,18,
         19,21,22,23,24,25,27,28,29,30,32,33,34,
         36,37,38,39,40,41,42,43,44,45,46,47,48,
         49,50,51,52,53,54,55,56,58,60,61,62,63,
         64,65,66,67,68,69,70,71,72,73,74,75,76,
         77,78,79,81,82,83,84,85,87,89
    ]

    '''
        Input record schema
        schema: n-m (o) -- xxx
            n - position from
            m - position to
            o - number of characters
            xxx - description
        1. 1-19 (19) -- reserved positions
        2. 20 (1) -- resident status
        3. 21-60 (40) -- reserved positions
        4. 61-62 (2) -- education code (1989 revision)
        5. 63 (1) -- education code (2003 revision)
        6. 64 (1) -- education reporting flag
        7. 65-66 (2) -- month of death
        8. 67-68 (2) -- reserved positions
        9. 69 (1) -- sex
        10. 70 (1) -- age: 1-years, 2-months, 4-days, 5-hours, 6-minutes, 9-not stated
        11. 71-73 (3) -- number of units (years, months etc)
        12. 74 (1) -- age substitution flag (if the age reported in positions 70-74 is calculated using dates of birth and death)
        13. 75-76 (2) -- age recoded into 52 categories
        14. 77-78 (2) -- age recoded into 27 categories
        15. 79-80 (2) -- age recoded into 12 categories
        16. 81-82 (2) -- infant age recoded into 22 categories
        17. 83 (1) -- place of death
        18. 84 (1) -- marital status
        19. 85 (1) -- day of the week of death
        20. 86-101 (16) -- reserved positions
        21. 102-105 (4) -- current year
        22. 106 (1) -- injury at work
        23. 107 (1) -- manner of death
        24. 108 (1) -- manner of disposition
        25. 109 (1) -- autopsy
        26. 110-143 (34) -- reserved positions
        27. 144 (1) -- activity code
        28. 145 (1) -- place of injury
        29. 146-149 (4) -- ICD code
        30. 150-152 (3) -- 358 cause recode
        31. 153 (1) -- reserved position
        32. 154-156 (3) -- 113 cause recode
        33. 157-159 (3) -- 130 infant cause recode
        34. 160-161 (2) -- 39 cause recode
        35. 162 (1) -- reserved position
        36. 163-164 (2) -- number of entity-axis conditions
        37-56. 165-304 (140) -- list of up to 20 conditions
        57. 305-340 (36) -- reserved positions
        58. 341-342 (2) -- number of record axis conditions
        59. 343 (1) -- reserved position
        60-79. 344-443 (100) -- record axis conditions
        80. 444 (1) -- reserve position
        81. 445-446 (2) -- race
        82. 447 (1) -- bridged race flag
        83. 448 (1) -- race imputation flag
        84. 449 (1) -- race recode (3 categories)
        85. 450 (1) -- race recode (5 categories)
        86. 461-483 (33) -- reserved positions
        87. 484-486 (3) -- Hispanic origin
        88. 487 (1) -- reserved
        89. 488 (1) -- Hispanic origin/race recode
     '''

    record_split = re\
        .compile(
            r'([\s]{19})([0-9]{1})([\s]{40})([0-9\s]{2})([0-9\s]{1})([0-9]{1})([0-9]{2})' + 
            r'([\s]{2})([FM]{1})([0-9]{1})([0-9]{3})([0-9\s]{1})([0-9]{2})([0-9]{2})' + 
            r'([0-9]{2})([0-9\s]{2})([0-9]{1})([SMWDU]{1})([0-9]{1})([\s]{16})([0-9]{4})' +
            r'([YNU]{1})([0-9\s]{1})([BCOU]{1})([YNU]{1})([\s]{34})([0-9\s]{1})([0-9\s]{1})' +
            r'([A-Z0-9\s]{4})([0-9]{3})([\s]{1})([0-9\s]{3})([0-9\s]{3})([0-9\s]{2})([\s]{1})' + 
            r'([0-9\s]{2})([A-Z0-9\s]{7})([A-Z0-9\s]{7})([A-Z0-9\s]{7})([A-Z0-9\s]{7})' + 
            r'([A-Z0-9\s]{7})([A-Z0-9\s]{7})([A-Z0-9\s]{7})([A-Z0-9\s]{7})([A-Z0-9\s]{7})' + 
            r'([A-Z0-9\s]{7})([A-Z0-9\s]{7})([A-Z0-9\s]{7})([A-Z0-9\s]{7})([A-Z0-9\s]{7})' + 
            r'([A-Z0-9\s]{7})([A-Z0-9\s]{7})([A-Z0-9\s]{7})([A-Z0-9\s]{7})([A-Z0-9\s]{7})' + 
            r'([A-Z0-9\s]{7})([\s]{36})([A-Z0-9\s]{2})([\s]{1})([A-Z0-9\s]{5})([A-Z0-9\s]{5})' + 
            r'([A-Z0-9\s]{5})([A-Z0-9\s]{5})([A-Z0-9\s]{5})([A-Z0-9\s]{5})([A-Z0-9\s]{5})' + 
            r'([A-Z0-9\s]{5})([A-Z0-9\s]{5})([A-Z0-9\s]{5})([A-Z0-9\s]{5})([A-Z0-9\s]{5})' + 
            r'([A-Z0-9\s]{5})([A-Z0-9\s]{5})([A-Z0-9\s]{5})([A-Z0-9\s]{5})([A-Z0-9\s]{5})' + 
            r'([A-Z0-9\s]{5})([A-Z0-9\s]{5})([A-Z0-9\s]{5})([\s]{1})([0-9\s]{2})([0-9\s]{1})' + 
            r'([0-9\s]{1})([0-9\s]{1})([0-9\s]{1})([\s]{33})([0-9\s]{3})([0-9\s]{1})([0-9\s]{1})')
    try:
        rs = np.array(record_split.split(row))[selected_indices]
    except:
        rs = np.array(['-99'] * len(selected_indices))
    return rs
#     return record_split.split(row)

## 데이터를 변형하기 

In [16]:
data_from_file_conv = data_from_file.map(extractInformation)
data_from_file_conv.map(lambda row: row).take(1)

[array(['1', '  ', '2', '1', '01', 'M', '1', '087', ' ', '43', '23', '11',
        '  ', '4', 'M', '4', '2014', 'U', '7', 'C', 'N', ' ', ' ', 'I64 ',
        '238', '070', '   ', '24', '01', '11I64  ', '       ', '       ',
        '       ', '       ', '       ', '       ', '       ', '       ',
        '       ', '       ', '       ', '       ', '       ', '       ',
        '       ', '       ', '       ', '       ', '       ', '01',
        'I64  ', '     ', '     ', '     ', '     ', '     ', '     ',
        '     ', '     ', '     ', '     ', '     ', '     ', '     ',
        '     ', '     ', '     ', '     ', '     ', '     ', '01', ' ',
        ' ', '1', '1', '100', '6'], dtype='<U40')]

## 특정 데이터를 변환해서 가져오기 

In [17]:
data_2014 = data_from_file_conv.map(lambda row: int(row[16]))
data_2014.take(10)

[2014, 2014, 2014, 2014, 2014, 2014, 2014, 2014, 2014, -99]

In [18]:
data_2014

PythonRDD[6] at RDD at PythonRDD.scala:53

## 변환된 데이터에 액션 실행하기

In [19]:
a = data_2014.collect()

In [20]:
len(a)

2631171

In [22]:
type(a)

list

In [25]:
a[0], a[9]

(2014, -99)

In [28]:
a[10]

2014

## 16번째 데이터를 처리

In [21]:
data_2014_2 = data_from_file_conv.map(lambda row: (row[16], int(row[16])))
data_2014_2.take(10)

[('2014', 2014),
 ('2014', 2014),
 ('2014', 2014),
 ('2014', 2014),
 ('2014', 2014),
 ('2014', 2014),
 ('2014', 2014),
 ('2014', 2014),
 ('2014', 2014),
 ('-99', -99)]

## 특정 내용만 추출함

In [26]:
data_filtered = data_from_file_conv.filter(lambda row: row[5] == 'F' and row[21] == '0')
data_filtered.count()

6

In [29]:
data_filtered

PythonRDD[10] at RDD at PythonRDD.scala:53

## 자료형이 다를 경우 제외 시키고 처리함 

In [27]:
data_2014_flat = data_from_file_conv.flatMap(lambda row: (row[16], int(row[16]) + 1))
data_2014_flat.take(10)

['2014', 2015, '2014', 2015, '2014', 2015, '2014', 2015, '2014', 2015]

In [30]:
data_2014_flat

PythonRDD[11] at RDD at PythonRDD.scala:53

In [31]:
distinct_gender = data_from_file_conv.map(lambda row: row[5]).distinct().collect()


['-99', 'M', 'F']

In [32]:
distinct_gender

['-99', 'M', 'F']

# 6. 새로운 RDD 생성하기

In [33]:
data_reduce = sc.parallelize([1, 2, .5, .1, 5, .2], 3)
data_reduce.reduce(lambda x, y: x / y)

0.004

In [34]:
data_reduce

ParallelCollectionRDD[17] at parallelize at PythonRDD.scala:195

In [35]:
data_reduce.count()

6

In [36]:
ar = data_reduce.collect()

In [37]:
ar

[1, 2, 0.5, 0.1, 5, 0.2]

In [38]:
len(ar)

6

In [39]:
data_key = sc.parallelize([('a', 4),('b', 3),('c', 2),('a', 8),('d', 2),('b', 1),('d', 3)],4)
data_key.reduceByKey(lambda x, y: x + y).collect()

[('b', 4), ('c', 2), ('a', 12), ('d', 5)]

In [40]:
data_key

ParallelCollectionRDD[20] at parallelize at PythonRDD.scala:195

## 파일을 저장하기

In [41]:
data_key.saveAsTextFile('data_key.txt')

## 텍스트 파일을 읽고  정규표현식에 따라 분리하기

In [42]:
def parseInput(row):
    import re
    
    pattern = re.compile(r'\(\'([a-z])\', ([0-9])\)')
    row_split = pattern.split(row)
    
    return (row_split[1], int(row_split[2]))

In [43]:
data_key_reread = sc \
    .textFile('data_key.txt') \
    .map(parseInput)
data_key_reread.collect()

[('a', 8), ('d', 2), ('b', 1), ('d', 3), ('b', 3), ('c', 2), ('a', 4)]

## foreach로 print 할 경우는 cli 창에 출력

In [44]:
def f(x): 
    print(x)

data_key.foreach(f)